In [95]:
from datetime import datetime,timedelta, timezone, date
import pytz

In [11]:
#first avail DATETIME '2019-02-22T00:00:00'
# retrieved timestamp'2020-12-08 18:27:52 UTC'


events= ['Wed Mar 17', 'Thu Mar 18', 'Fri Mar 19']




first_event_date = datetime.strptime(events[0], '%a %b %d').replace(year=now.year)
print(first_event_date.strftime('%Y-%m-%d'))
print(datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S.%f %Z'))

date = first_event_date.replace(tzinfo=offset)

2021-03-17
2021-03-17 19:51:48.058036 UTC


In [12]:
date

datetime.datetime(2021, 3, 17, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600)))

In [13]:
first_event_date

datetime.datetime(2021, 3, 17, 0, 0)

In [14]:
timedelta(hours=3).total_seconds() > abs((now-date).total_seconds())

False

In [15]:
timedelta(days=.5) < date-now

False

In [16]:
timedelta(hours=3).total_seconds()

10800.0

In [17]:
from google.cloud import bigquery
from google.cloud import bigquery_storage

import google.auth

In [38]:
def retreive_past_dates():
    credentials, your_project_id = google.auth.default(
        scopes=["https://www.googleapis.com/auth/cloud-platform"]
    )
    print(your_project_id, credentials)
    # Make clients.
    bqclient = bigquery.Client(credentials=credentials, project=your_project_id,)
    bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

    query_string = """
    SELECT * 
    FROM `vaccine-volunteering.dates_accessed.dates`
    """

    dataframe = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    return dataframe

In [50]:
import pandas as pd

In [88]:
old_events = retreive_past_dates()

vaccine-volunteering <google.oauth2.credentials.Credentials object at 0x7fac4b3490a0>


In [91]:
old_events = old_events.groupby('date').max()

In [92]:
old_events.index.values

array([datetime.date(2021, 3, 17), datetime.date(2021, 3, 18),
       datetime.date(2021, 3, 19)], dtype=object)

In [93]:
def str_to_datetime(date_str):
    return datetime.strptime(date_str, '%a %b %d').date()

def date_to_bigquery(date):
    return date.strftime('%Y-%m-%d')

def timestamp_to_bigquery(date):
    return date.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S.%f %Z')

In [94]:
offset = timezone(timedelta(hours=-8))
now = datetime.now(offset)

events_dt = [str_to_datetime(date).replace(year=now.year) for date in events]

In [68]:
events_dt

[datetime.date(2021, 3, 17),
 datetime.date(2021, 3, 18),
 datetime.date(2021, 3, 19)]

In [76]:
old_events.loc[events_dt[0]].last_accessed

Timestamp('2021-03-17 20:02:21.002978+0000', tz='UTC')

In [84]:
datetime.now(pytz.UTC)

datetime.datetime(2021, 3, 17, 20, 16, 56, 266922, tzinfo=<UTC>)

In [87]:
new_events = []
for event in events_dt:
    if (event not in old_events.index.values) or \
    timedelta(hours=3).total_seconds() < abs((datetime.now(pytz.UTC)-old_events.loc[event].last_accessed).total_seconds()):
        print('time check')
        new_events.append({u"date":event, u"last_accessed":timestamp_to_bigquery(datetime.now())})
        

In [ ]:
def upload_results()

In [36]:
client = bigquery.Client()


In [37]:
errors = client.insert_rows_json('vaccine-volunteering.dates_accessed.dates', new_events)  # Make an API request.
if errors == []:
    print("New rows have been added.")
else:
    print("Encountered errors while inserting rows: {}".format(errors))

New rows have been added.
